**Script para añadir el nombre del gen al accesion number de cada anotación**
**Javier Gutiérrez Corral 09/12/2023**

In [1]:
#Se importan los paquetes necesarios

import pandas as pd
from Bio import Entrez

In [2]:
#Se abre el archivo con las anotaciones

columns=["Query_accesion","Target_accesion","Sequence Identity","Length","Mismatches","Gap_openings",
         "Query_start","Query_end","Target_start","Target_end","E-value","Bit_score"]
file2 = pd.read_csv(r"matchesNR.tsv",
                   sep ="\t",skiprows=0)
file2.columns=columns
file2.head()

,Query_accesion,Target_accesion,Sequence Identity,Length,Mismatches,Gap_openings,Query_start,Query_end,Target_start,Target_end,E-value,Bit_score
0,cluster_6,XP_047101492.1,62.1,3471,1119,46,3,9896,4519,7964,0.0,3455.0
1,cluster_6,XP_049843621.1,61.6,3455,1137,47,3,9896,3569,6998,0.0,3428.0
2,cluster_6,XP_049843616.1,61.6,3455,1137,47,3,9896,3652,7081,0.0,3428.0
3,cluster_6,XP_049843618.1,61.6,3455,1137,47,3,9896,3652,7081,0.0,3428.0
4,cluster_6,XP_049843617.1,61.6,3455,1137,47,3,9896,3647,7076,0.0,3428.0


In [3]:
#Se crean listas vacias en las que se irá añadiendo las variables de interés

L=[]
Lc=[]
Ln=[]
Le=[]

In [3]:
#De esta forma se pueden  cargar resultados previos para seguir donde se había quedado

file3 = pd.read_csv(r"api_3",
                   sep ="\t",skiprows=0)

In [5]:
#Comprobar cuáles son los últimos que se han añadido

file3.tail()

,cluster,Accesion,Anotation,e-value
8548,cluster_18196,XP_049809612,sialin-like [Schistocerca nitens],2.860000e-294
8549,cluster_18197,XP_047001732,N6-adenosine-methyltransferase catalytic subun...,0.000000e+00
8550,cluster_18198,KAF8763103,putative transposase-like protein like [Argiop...,8.110000e-75
8551,cluster_18200,XP_049791718,F-box/WD repeat-containing protein 2-like isof...,0.000000e+00
8552,cluster_18202,XP_049937658,cytochrome P450 4C1-like isoform X2 [Schistoce...,6.510000e-153


In [6]:
#Se añade en las listas que se utilizan en la llamada de la api los datos que se habían guardado

for i in file3["cluster"]:
    Lc.append(i)
for i in file3["Accesion"]:
    L.append(i)
for i in file3["Anotation"]:
    Ln.append(i)
for i in file3["e-value"]:
    Le.append(i)

In [7]:
#Otra forma de ver cuál era el último
print(L[-1],Lc[-1])

XP_049937658 cluster_18202


In [ ]:
'''
Script para acceder a la API de Entrez que permite recuperar la información de cada ID, dejando solo los que
estén caracterizados
Este es el bloque de código que se utiliza cuando no hay datos de busquedas anteriores sin acabar
'''

Entrez.email="Javier.gutierrezcorral@estudiante.uam.es"

for i,j,e in zip(file2["Target_accesion"],file2["Query_accesion"], file2["E-value"]):
    
    if j in Lc:
               pass
    else:
        handle = Entrez.efetch(db="protein", id=i, retmode="xml")
        records = Entrez.parse(handle)
        
        for record in records:
            if "uncharacterized" not in record["GBSeq_definition"] and "Uncharacterized" not in record["GBSeq_definition"]\
            and "hypothetical" not in record["GBSeq_definition"] and "Hypothetical" not in record["GBSeq_definition"]\
            and "unnamed" not in record["GBSeq_definition"] and "Unnamed" not in record["GBSeq_definition"]:
                
                print(i,j,(record["GBSeq_definition"]))
                L.append(record["GBSeq_locus"])
                Lc.append(j)
                Ln.append(record["GBSeq_definition"])
                Le.append(e)
                

        handle.close()

In [ ]:
'''
Script para acceder a la API de Entrez que permite recuperar la información de cada ID, dejando solo los que
estén caracterizados. Esta versión permite acceder a los resultados previos
'''

flag="False"
Entrez.email="Javier.gutierrezcorral@estudiante.uam.es"

for i,j,e in zip(file2["Target_accesion"],file2["Query_accesion"], file2["E-value"]):
    
    if str(j) == str(Lc[-1]):
        flag="True"
    elif j in Lc:
               pass
    elif flag == "True":
        handle = Entrez.efetch(db="protein", id=i, retmode="xml")
        records = Entrez.parse(handle)
        
        for record in records:
            if "uncharacterized" not in record["GBSeq_definition"] and "Uncharacterized" not in record["GBSeq_definition"]\
            and "hypothetical" not in record["GBSeq_definition"] and "Hypothetical" not in record["GBSeq_definition"]\
            and "unnamed" not in record["GBSeq_definition"] and "Unnamed" not in record["GBSeq_definition"]\
            and "unknown function" not in record["GBSeq_definition"] :
                print(i,j,(record["GBSeq_definition"]))
                
                L.append(record["GBSeq_locus"])
                Lc.append(j)
                Ln.append(record["GBSeq_definition"])
                Le.append(e)
                

        handle.close()

In [9]:
#Se guarda el resultado en un dataframe que luego se guardará en un archivo
df=pd.DataFrame()

In [10]:
df["cluster"]=Lc
df["Accesion"]=L
df["Anotation"]=Ln
df["e-value"]=Le

In [11]:
df.tail()

,cluster,Accesion,Anotation,e-value
10496,cluster_21227,XP_049802074,erlin-1-like [Schistocerca nitens],2.520000e-215
10497,cluster_21228,XP_049847062,P-selectin-like [Schistocerca gregaria],0.000000e+00
10498,cluster_21229,XP_047002870,geminin [Schistocerca americana],1.080000e-93
10499,cluster_21231,BAM63554,FAS-associated factor 1 [Locusta migratoria ma...,0.000000e+00
10500,cluster_21233,ATX63054,autophagy-related protein 2 [Locusta migratoria],0.000000e+00


In [12]:
df.to_csv(r"api_final",sep="\t",index=False,header=True)